In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [8]:
#Check Memory adapted from : https://stackoverflow.com/questions/48750199/google-colaboratory-misleading-information-about-its-gpu-only-5-ram-available
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " I Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 9.7 GB  I Proc size: 3.3 GB
GPU RAM Free: 562MB | Used: 10877MB | Util  95% | Total 11439MB


In [3]:
# Test if GPU is active
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [4]:
# Load libraries
import numpy as np
import pandas as pd
import io
import zipfile
from skimage import color
from matplotlib import pyplot
import keras
import h5py    
from keras.preprocessing.image import ImageDataGenerator
from keras import applications
import scipy.misc
from keras.models import load_model
from os import listdir
from keras.models import Sequential


%matplotlib inline

Using TensorFlow backend.


In [0]:
# Import & load our model
model_file = drive.CreateFile({'id': '1qO_PUBl6GL5SUP6hm1dDKMBQZ6X7ZZ10'})
model_file.GetContentFile('Final_Model.h5')
model = load_model('Final_Model.h5')

In [0]:
# Import test set
test_data_downloaded = drive.CreateFile({'id': '1mJEyhlbk1l52bCN8CFeVKwZS1yhcM9Oo'})
test_data_downloaded.GetContentFile('test_set.zip')  
test_data = zipfile.ZipFile('test_set.zip', 'r')
test_data.extractall('test_set/test_set') 


In [0]:
# Import test labels
test_label_downloaded = drive.CreateFile({'id': '1Fgtp2b-POelymbGYjKtioodOpCR_APe-'})
test_label_downloaded.GetContentFile('test.txt')
test_labels = pd.read_csv('test.txt', sep=" ", header=None)



In [0]:

# Convert test images into RGB arrays ###
test_images = []
for i in range (0, len(test_labels[0][:])):
  image = scipy.misc.imread('test_set/test_set/' + test_labels[0][i], flatten=False, mode='RGB')
  test_images.append(image.astype('uint8'))
  del image


In [31]:
# Import VGG16 without fully-connected layer
vgg16_model = applications.VGG16(include_top=False, weights='imagenet')

# Samples count and directory
test_sample_count = len(test_labels[0][:])
test_directory = ('test_set')

# Create a data generator for test set
datagen = ImageDataGenerator(rescale=1. / 255)
generator = datagen.flow_from_directory(
    test_directory,
    target_size=(150, 150), 
    batch_size=31,
    color_mode='rgb',
    class_mode=None,
    shuffle=False)

# Run the data through VGG16's convolution layers once to learn the features
test_features = vgg16_model.predict_generator(
    generator, test_sample_count // 31)

58892288/58889256 [==============================] - 6s 0us/step
Found 37696 images belonging to 1 classes.


In [0]:
# Define validation data for our model
X_test = test_features

# Predict outputs
y_pred = np.argmax(model.predict(X_test), axis=1)

# Print outputs
print('Predicted labels:', y_pred)


In [0]:

# Print output to txt file
with open("test.txt", "w") as output_file:
  for i in range (len(test_labels[0][:])):
    print(test_labels[0][i] + ' ' + str(y_pred[i]), file=output_file)
    
# Upload the output file to drive
output_file = drive.CreateFile({'title':'test.txt'})
output_file.SetContentFile('test.txt')
output_file.Upload()

In [40]:
# Print the first 100 predicted lavels
y_pred[0:100]

array([36, 23,  0, 24, 10,  1,  6, 22, 54, 12, 29, 53,  3, 22, 10,  4,  3,
       52, 10, 39, 12, 60, 38, 30, 48, 30, 53,  2, 49, 29, 32,  9, 57, 28,
       52, 25, 30, 28, 59, 35, 57,  7, 47, 25, 12, 29, 56, 26, 42,  9, 52,
       58, 34, 15,  3, 54, 13, 60, 25, 33, 58, 54, 48, 13, 50, 13, 23, 56,
       22, 44, 47, 52,  8, 58, 36, 44, 52, 34,  7,  0, 26, 41, 45, 19, 30,
       50, 44, 45, 16, 42, 27, 12, 61, 31, 35, 39, 48, 56, 33, 44])